# 제품 이상여부 판별 프로젝트


### 데이터 읽어오기


In [67]:
import os
from pprint import pprint

import optuna
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [68]:
import pandas as pd

RANDOM_STATE = 110

train_data = pd.read_csv('./data/train_data_0827.csv')
test_data = pd.read_csv('./data/test_data_0827.csv')

In [69]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Receip_No_encoded',
    'Equipment_same_num',
    'PalletID_Collect_Result_encoded',
    'Production_Qty_Collect_Result',
    'WorkMode Collect Result'
]

In [70]:
# 전체 공통 변수
### train
var_all_train = [
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

### test
var_all_test = [
    'Set ID',
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

In [71]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [72]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [73]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [74]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]

In [75]:
# 각 DataFrame의 칼럼 수 계산
num_columns_train_data = train_data.shape[1]
num_columns_train_data_dam = train_data_dam.shape[1]
num_columns_train_data_autoclave = train_data_autoclave.shape[1]
num_columns_train_data_fill1 = train_data_fill1.shape[1]
num_columns_train_data_fill2 = train_data_fill2.shape[1]

num_columns_test_data = test_data.shape[1]
num_columns_test_data_dam = test_data_dam.shape[1]
num_columns_test_data_autoclave = test_data_autoclave.shape[1]
num_columns_test_data_fill1 = test_data_fill1.shape[1]
num_columns_test_data_fill2 = test_data_fill2.shape[1]

# 각 DataFrame의 칼럼 수 출력
print("----train data-----")
print(f"train_data DataFrame의 칼럼 수: {num_columns_train_data}")
print(f"train_data_dam DataFrame의 칼럼 수: {num_columns_train_data_dam}")
print(f"train_data_autoclave DataFrame의 칼럼 수: {num_columns_train_data_autoclave}")
print(f"train_data_fill1 DataFrame의 칼럼 수: {num_columns_train_data_fill1}")
print(f"train_data_fill2 DataFrame의 칼럼 수: {num_columns_train_data_fill2}")
print("----test data-----")
print(f"test_data DataFrame의 칼럼 수: {num_columns_test_data}")
print(f"test_data_dam DataFrame의 칼럼 수: {num_columns_test_data_dam}")
print(f"test_data_autoclave DataFrame의 칼럼 수: {num_columns_test_data_autoclave}")
print(f"test_data_fill1 DataFrame의 칼럼 수: {num_columns_test_data_fill1}")
print(f"test_data_fill2 DataFrame의 칼럼 수: {num_columns_test_data_fill2}")

----train data-----
train_data DataFrame의 칼럼 수: 40
train_data_dam DataFrame의 칼럼 수: 23
train_data_autoclave DataFrame의 칼럼 수: 8
train_data_fill1 DataFrame의 칼럼 수: 14
train_data_fill2 DataFrame의 칼럼 수: 14
----test data-----
test_data DataFrame의 칼럼 수: 41
test_data_dam DataFrame의 칼럼 수: 24
test_data_autoclave DataFrame의 칼럼 수: 9
test_data_fill1 DataFrame의 칼럼 수: 15
test_data_fill2 DataFrame의 칼럼 수: 15


---

## Optuna

스레스홀드 0.3으로 맞춘상태에서 튜닝 진행한 것

In [76]:
RANDOM_STATE = 110
THRESHOLD = 0.3

In [77]:
# 'Normal'과 'AbNormal'을 숫자로 변환
train_data_dam['target'] = train_data_dam['target'].map({'Normal': 0, 'AbNormal': 1})

# 스레드홀드 설정
THRESHOLD = 0.3

def objectiveRandomForestClassifier(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'n_estimators' : trial.suggest_int('n_estimators', 800, 3000),
        'max_depth' : trial.suggest_int('max_depth', 20, 100),
        'min_samples_split' : trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf' : trial.suggest_int('min_samples_leaf', 1, 8),
        'criterion' : trial.suggest_categorical("criterion", ["entropy",]),
        'class_weight' : trial.suggest_categorical("class_weight", ["balanced"]),
        'random_state': RANDOM_STATE
    }
    
    model = RandomForestClassifier(**param)
    model.fit(x_tr, y_tr)
    pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data_dam.drop("target", axis=1),
    train_data_dam["target"],
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study.optimize(lambda trial: objectiveRandomForestClassifier(trial, x_train, y_train, x_val, y_val), n_trials=300)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

/tmp/ipykernel_126/706836933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_dam['target'] = train_data_dam['target'].map({'Normal': 0, 'AbNormal': 1})
[I 2024-08-28 02:51:05,219] A new study created in memory with name: no-name-c57dd17a-a655-44dc-99d0-890011588390
[I 2024-08-28 02:51:28,522] Trial 0 finished with value: 0.175958749597164 and parameters: {'n_estimators': 1055, 'max_depth': 73, 'min_samples_split': 9, 'min_samples_leaf': 5, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.175958749597164.
[I 2024-08-28 02:52:22,290] Trial 1 finished with value: 0.1627027027027027 and parameters: {'n_estimators': 2295, 'max_depth': 75, 'min_samples_split': 20, 'min_samples_leaf': 6, 'criterion': 'ent

[I 2024-08-28 03:43:46,381] Trial 28 finished with value: 0.21188630490956076 and parameters: {'n_estimators': 2381, 'max_depth': 93, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 26 with value: 0.2144702842377261.
[I 2024-08-28 03:44:54,652] Trial 29 finished with value: 0.17670682730923695 and parameters: {'n_estimators': 1995, 'max_depth': 89, 'min_samples_split': 2, 'min_samples_leaf': 3, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 26 with value: 0.2144702842377261.
[I 2024-08-28 03:46:49,369] Trial 30 finished with value: 0.18146718146718144 and parameters: {'n_estimators': 2110, 'max_depth': 62, 'min_samples_split': 3, 'min_samples_leaf': 2, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 26 with value: 0.2144702842377261.
[I 2024-08-28 03:49:27,230] Trial 31 finished with value: 0.20886699507389164 and parameters: {'n_estimators': 2805, 'max_depth': 95, 'min_samples_spli

[I 2024-08-28 04:26:59,561] Trial 57 finished with value: 0.17131272174353773 and parameters: {'n_estimators': 949, 'max_depth': 88, 'min_samples_split': 8, 'min_samples_leaf': 2, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 48 with value: 0.21854304635761587.
[I 2024-08-28 04:28:12,239] Trial 58 finished with value: 0.21125370187561698 and parameters: {'n_estimators': 1416, 'max_depth': 54, 'min_samples_split': 4, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 48 with value: 0.21854304635761587.
[I 2024-08-28 04:29:17,282] Trial 59 finished with value: 0.19912663755458515 and parameters: {'n_estimators': 1747, 'max_depth': 84, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 48 with value: 0.21854304635761587.
[I 2024-08-28 04:29:57,227] Trial 60 finished with value: 0.17612874385337504 and parameters: {'n_estimators': 1165, 'max_depth': 80, 'min_samples_sp

[I 2024-08-28 05:03:29,925] Trial 86 finished with value: 0.1834625322997416 and parameters: {'n_estimators': 1398, 'max_depth': 86, 'min_samples_split': 3, 'min_samples_leaf': 2, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 48 with value: 0.21854304635761587.
[I 2024-08-28 05:04:52,837] Trial 87 finished with value: 0.21167161226508407 and parameters: {'n_estimators': 1480, 'max_depth': 98, 'min_samples_split': 4, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 48 with value: 0.21854304635761587.
[I 2024-08-28 05:05:47,124] Trial 88 finished with value: 0.17274826789838335 and parameters: {'n_estimators': 1009, 'max_depth': 89, 'min_samples_split': 10, 'min_samples_leaf': 2, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 48 with value: 0.21854304635761587.
[I 2024-08-28 05:07:10,380] Trial 89 finished with value: 0.2006980802792321 and parameters: {'n_estimators': 1317, 'max_depth': 91, 'min_samples_sp

[I 2024-08-28 05:38:30,868] Trial 115 finished with value: 0.2175824175824176 and parameters: {'n_estimators': 1259, 'max_depth': 44, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 100 with value: 0.21878453038674037.
[I 2024-08-28 05:39:47,033] Trial 116 finished with value: 0.21209117938553024 and parameters: {'n_estimators': 1286, 'max_depth': 45, 'min_samples_split': 4, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 100 with value: 0.21878453038674037.
[I 2024-08-28 05:40:54,466] Trial 117 finished with value: 0.2172949002217295 and parameters: {'n_estimators': 1066, 'max_depth': 41, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 100 with value: 0.21878453038674037.
[I 2024-08-28 05:42:03,500] Trial 118 finished with value: 0.1801685029163966 and parameters: {'n_estimators': 1246, 'max_depth': 45, 'min_sampl

...

[I 2024-08-28 06:53:23,658] Trial 202 finished with value: 0.21878453038674037 and parameters: {'n_estimators': 1037, 'max_depth': 54, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 121 with value: 0.2195121951219512.
[I 2024-08-28 06:54:22,943] Trial 203 finished with value: 0.21878453038674037 and parameters: {'n_estimators': 1020, 'max_depth': 47, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 121 with value: 0.2195121951219512.
[I 2024-08-28 06:55:17,845] Trial 204 finished with value: 0.2172949002217295 and parameters: {'n_estimators': 921, 'max_depth': 46, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 121 with value: 0.2195121951219512.
[I 2024-08-28 06:56:12,496] Trial 205 finished with value: 0.20689655172413796 and parameters: {'n_estimators': 1014, 'max_depth': 48, 'min_samples

[I 2024-08-28 07:22:06,392] Trial 231 finished with value: 0.21878453038674037 and parameters: {'n_estimators': 1073, 'max_depth': 57, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 121 with value: 0.2195121951219512.
[I 2024-08-28 07:22:50,533] Trial 232 finished with value: 0.21854304635761587 and parameters: {'n_estimators': 1053, 'max_depth': 54, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 121 with value: 0.2195121951219512.
[I 2024-08-28 07:23:30,499] Trial 233 finished with value: 0.16644532979194337 and parameters: {'n_estimators': 1006, 'max_depth': 53, 'min_samples_split': 15, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 121 with value: 0.2195121951219512.
[I 2024-08-28 07:24:32,685] Trial 234 finished with value: 0.21854304635761587 and parameters: {'n_estimators': 1084, 'max_depth': 56, 'min_samp

[I 2024-08-28 07:43:05,455] Trial 260 finished with value: 0.20886699507389164 and parameters: {'n_estimators': 964, 'max_depth': 42, 'min_samples_split': 4, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 121 with value: 0.2195121951219512.
[I 2024-08-28 07:43:32,752] Trial 261 finished with value: 0.21782178217821785 and parameters: {'n_estimators': 1094, 'max_depth': 55, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 121 with value: 0.2195121951219512.
[I 2024-08-28 07:43:58,805] Trial 262 finished with value: 0.2144702842377261 and parameters: {'n_estimators': 1031, 'max_depth': 49, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 121 with value: 0.2195121951219512.
[I 2024-08-28 07:44:24,269] Trial 263 finished with value: 0.1753130590339893 and parameters: {'n_estimators': 1122, 'max_depth': 53, 'min_samples_

[I 2024-08-28 07:58:23,901] Trial 289 finished with value: 0.211340206185567 and parameters: {'n_estimators': 1077, 'max_depth': 56, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 121 with value: 0.2195121951219512.
[I 2024-08-28 07:58:46,462] Trial 290 finished with value: 0.17183528215556684 and parameters: {'n_estimators': 918, 'max_depth': 61, 'min_samples_split': 12, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 121 with value: 0.2195121951219512.
[I 2024-08-28 07:59:16,010] Trial 291 finished with value: 0.21830209481808158 and parameters: {'n_estimators': 1109, 'max_depth': 54, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 121 with value: 0.2195121951219512.
[I 2024-08-28 07:59:45,112] Trial 292 finished with value: 0.21878453038674037 and parameters: {'n_estimators': 1028, 'max_depth': 48, 'min_samples

Best trial: score 0.2195121951219512, 
params {'n_estimators': 1113, 'max_depth': 43, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}


Best trial: score 0.2195121951219512, 
params {'n_estimators': 1113, 'max_depth': 43, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}